In [1]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import copy
import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMRegressor
import joblib

# Read data

In [2]:
rawdata_dir = 'rawdata'
data_dir = 'data'

In [3]:
sales = pd.read_csv(os.path.join(rawdata_dir, 'sales_train_evaluation.csv'))
sales.name = 'sales'
calendar = pd.read_csv(os.path.join(rawdata_dir, 'calendar.csv'))
calendar.name = 'calendar'
prices = pd.read_csv(os.path.join(rawdata_dir, 'sell_prices.csv'))
prices.name = 'prices'

Since, the validation data is now available for the days 1914-1941, Adding zero sales for days: d_1942 - d_1969(Test)

In [4]:
#Add zero sales for the remaining days 1942-1969
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

# Downcast

In [5]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# Melt data

In [6]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

## Combine data
Combine price data from prices dataframe and days data from calendar dataset.

In [7]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

# Feature Engineering

## Label Encoding

In [8]:
#Store the categories along with their codes
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [9]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
df.drop('date',axis=1,inplace=True)

## Introduce lags

In [10]:
#Introduce lags
def make_lags(df, lags):
    for lag in lags:
        df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
        
    return df

In [11]:
lags = [1,2,3,6,12,24,36]
df = make_lags(df, lags)

## make new features

In [21]:
def make_new_feature(df, d_train_end = df['d'].max()):
    # copy sold
    sold_tmp = copy.copy(df['sold'])
    
    # embed NaN
    df['sold'][df['d'] < d_train_end] = np.nan
    
    # make feature using sold
    df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
    df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
    df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
    df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
    df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
    df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
    df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    
    df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    
    df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
    df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
    df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
    df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)
    
    # restore sold
    df['sold'] = sold_tmp
    
    return df

## Save data (Use this after CV)

In [22]:
df_all = make_new_feature(df)
df_all = df_all[df_all['d'] >= 36]

In [23]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58967660 entries, 1067150 to 60034809
Data columns (total 43 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int16  
 1   item_id                   int16  
 2   dept_id                   int8   
 3   cat_id                    int8   
 4   store_id                  int8   
 5   state_id                  int8   
 6   d                         int16  
 7   sold                      float64
 8   wm_yr_wk                  int16  
 9   weekday                   int8   
 10  wday                      int8   
 11  month                     int8   
 12  year                      int16  
 13  event_name_1              int8   
 14  event_type_1              int8   
 15  event_name_2              int8   
 16  event_type_2              int8   
 17  snap_CA                   int8   
 18  snap_TX                   int8   
 19  snap_WI                   int8   
 20  sell_price      

In [24]:
df_all.to_pickle(os.path.join(data_dir, 'data_all.pkl'))
del df_all
gc.collect();

# Cross Validation

## Parameter list

In [ ]:
# model = LGBMRegressor(
#     n_estimators=1000,
#     learning_rate=0.3,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     max_depth=8,
#     num_leaves=50,
#     min_child_weight=300
# )

## Remove evaluation from df

In [24]:
NUM_FOLD_DAYS = 28
VALID_END_DAY = 1941

In [ ]:
df = df[df['d'] <= VALID_END_DAY]

## CV Loop

In [ ]:
stores = sales.store_id.cat.codes.unique().tolist()
result_mat = np.zeros([1, len(stores)])   # (#All-possible-params-combination) x (#store)
fold_iter = 0

### LOOP: CV fold ###
while true
    d_valid_end = VALID_END_DAY - NUM_FOLD_DAYS * fold_iter
    d_train_end = VALID_END_DAY - NUM_FOLD_DAYS * (fold_iter + 1)
    
    df_fold = df[df['d'] <= d_valid_end]
    df_fold = make_new_feature(df_fold, d_train_end)   # train の情報だけを使って特徴量を作成
    
    fold_iter += 1
    

In [ ]:
def fit_each_store(data, params, d_train_end, d_valid_end, result_mat):
    stores = sales.store_id.cat.codes.unique().tolist()
    for store in stores:
        df = data[data['store_id']==store]

        #Split the data
        X_train, y_train = df[df['d'] <= d_train_end].drop('sold',axis=1), df[df['d'] <= d_train_end]['sold']
        X_valid, y_valid = df[(df['d'] > d_train_end) & (df['d'] <= d_valid_end)].drop('sold',axis=1), df[(df['d'] > d_train_end) & (df['d'] <= d_valid_end)]['sold']

        # Train
        model = LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.3,
            subsample=0.8,
            colsample_bytree=0.8,
            max_depth=8,
            num_leaves=50,
            min_child_weight=300
        )
        
        print('*****Prediction for Store: {}*****'.format(d_store_id[store]))
        model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],
                 eval_metric='rmse', verbose=20, early_stopping_rounds=20)
        valid_preds[X_valid.index] = model.predict(X_valid)
        eval_preds[X_test.index] = model.predict(X_test)
        filename = 'model'+str(d_store_id[store])+'.pkl'
        # save model
        joblib.dump(model, filename)
        del model, X_train, y_train, X_valid, y_valid
        gc.collect()
        

In [67]:
params = {
    'n_estimators': 1000,
    'learning_rate': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_depth': 8,
    'num_leaves': 50,
    'min_child_weight': 300
}

param_space = {
    'n_estimators': 1000,
    'learning_rate': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_depth': 8,
    'num_leaves': 50,
    'min_child_weight': 300
}

In [70]:
params['max_depth']

8